In [2]:
import pandas as pd

In [3]:
import sqlite3

In [4]:
import requests

In [7]:
conn = sqlite3.connect(':memory:') 
cursor = conn.cursor()
cursor.execute('CREATE TABLE students (id INTEGER, name TEXT, grade INTEGER)')
data = [(1, 'Alice', 85), (2, 'Bob', 90), (3, 'Charlie', 78), (4, 'David', 92)]
cursor.executemany('INSERT INTO students VALUES (?,?,?)', data)
conn.commit()

In [8]:
# data collection from a database
query = "SELECT * FROM students"
df_db = pd.read_sql_query(query, conn)
print(df_db)
conn.close()

   id     name  grade
0   1    Alice     85
1   2      Bob     90
2   3  Charlie     78
3   4    David     92


In [11]:
# data collection from an API

api_url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(api_url)

if response.status_code == 200:
    json_data = response.json()
    df_api = pd.DataFrame(json_data)
    df_api_clean = df_api[['id', 'name', 'email']]
    print(df_api_clean.head())
else:
    print("Failed to fetch API data")

   id              name                      email
0   1     Leanne Graham          Sincere@april.biz
1   2      Ervin Howell          Shanna@melissa.tv
2   3  Clementine Bauch         Nathan@yesenia.net
3   4  Patricia Lebsack  Julianne.OConner@kory.org
4   5  Chelsey Dietrich   Lucio_Hettinger@annie.ca


In [12]:
# data colletion from a sensor

sensor_data_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"

try:
    df_sensor = pd.read_csv(sensor_data_url, parse_dates=['Date'])
    print(df_sensor.head())
except Exception as e:
    print(f"Error: {e}")

        Date  Temp
0 1981-01-01  20.7
1 1981-01-02  17.9
2 1981-01-03  18.8
3 1981-01-04  14.6
4 1981-01-05  15.8


In [13]:
from bs4 import BeautifulSoup

In [14]:
# data collection by web scraping

url = "http://quotes.toscrape.com/"

try:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # The site holds quotes in <span class="text"> and authors in <small class="author">
    quotes = soup.find_all('span', class_='text')
    authors = soup.find_all('small', class_='author')

    scraped_data = []
    for quote, author in zip(quotes, authors):
        scraped_data.append({
            'Author': author.text,
            'Quote': quote.text
        })

    df_scraping = pd.DataFrame(scraped_data)
    print(df_scraping.head())

except Exception as e:
    print(f"Error: {e}")

            Author                                              Quote
0  Albert Einstein  “The world as we have created it is a process ...
1     J.K. Rowling  “It is our choices, Harry, that show what we t...
2  Albert Einstein  “There are only two ways to live your life. On...
3      Jane Austen  “The person, be it gentleman or lady, who has ...
4   Marilyn Monroe  “Imperfection is beauty, madness is genius and...
